# Spatial Analysis

Spatial neighborhood analysis, co-occurrence, and spatial statistics.

**Input:** Annotated data from 02_phenotyping.ipynb
**Output:** Spatial analysis results

In [ ]:
import sys
sys.path.append('..')
import numpy as np
import pandas as pd
import scanpy as sc
import squidpy as sq
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

sc.settings.verbosity = 3
DATA_DIR = Path('../data/processed')
FIGURES_DIR = Path('../figures/03_spatial_analysis')
FIGURES_DIR.mkdir(parents=True, exist_ok=True)
SAMPLE_NAME = 'phenocycler_sample_01'

In [ ]:
adata = sc.read_h5ad(DATA_DIR / f'{SAMPLE_NAME}_annotated.h5ad')
print(f'Loaded: {adata.shape[0]} cells')
if 'spatial' not in adata.obsm:
    print('ERROR: No spatial coordinates found!')
else:
    print(f'Spatial coordinates present: {adata.obsm["spatial"].shape}')

## Spatial Neighborhood Graph

In [ ]:
sq.gr.spatial_neighbors(adata, n_neighs=10, coord_type='generic', spatial_key='spatial')
print('Spatial neighborhood graph computed')

## Neighborhood Enrichment

In [ ]:
sq.gr.nhood_enrichment(adata, cluster_key='celltype')
sq.pl.nhood_enrichment(adata, cluster_key='celltype', save='_enrichment.png')
print('Neighborhood enrichment computed')

## Co-occurrence Analysis

In [ ]:
sq.gr.co_occurrence(adata, cluster_key='celltype')
sq.pl.co_occurrence(adata, cluster_key='celltype', clusters='celltype', save='_cooccurrence.png')

## Spatial Autocorrelation

In [ ]:
markers = adata.var_names[:10]
sq.gr.spatial_autocorr(adata, mode='moran', genes=markers)
print('Top spatially variable markers:')
print(adata.uns['moranI'].head(10))

## Ripley Statistics

In [ ]:
mode = 'L'
sq.gr.ripley(adata, cluster_key='celltype', mode=mode, spatial_key='spatial')
sq.pl.ripley(adata, cluster_key='celltype', mode=mode, save='_ripley.png')

## Spatial Visualization

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))
coords = adata.obsm['spatial']
celltype_colors = dict(zip(adata.obs['celltype'].cat.categories, 
                          plt.cm.tab20.colors[:len(adata.obs['celltype'].cat.categories)]))
for ct in adata.obs['celltype'].unique():
    mask = adata.obs['celltype'] == ct
    ax.scatter(coords[mask, 0], coords[mask, 1], 
               c=[celltype_colors[ct]], label=ct, s=1, alpha=0.6)
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax.set_title('Spatial Distribution of Cell Types')
plt.tight_layout()
plt.savefig(FIGURES_DIR / 'spatial_celltypes.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
output_file = DATA_DIR / f'{SAMPLE_NAME}_spatial_analysis.h5ad'
adata.write_h5ad(output_file)
print(f'Spatial analysis data saved to: {output_file}')